In [1]:
import h2o
import h2o.grid
from h2o.estimators.deeplearning import H2ODeepLearningEstimator
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.11" 2021-04-20; OpenJDK Runtime Environment (build 11.0.11+9-Ubuntu-0ubuntu2.20.04); OpenJDK 64-Bit Server VM (build 11.0.11+9-Ubuntu-0ubuntu2.20.04, mixed mode, sharing)
  Starting server from /home/gerd/Documents/Python/venvs/h2o/lib/python3.8/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp5vjpzbjj
  JVM stdout: /tmp/tmp5vjpzbjj/h2o_gerd_started_from_python.out
  JVM stderr: /tmp/tmp5vjpzbjj/h2o_gerd_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Europe/Vienna
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.2
H2O_cluster_version_age:,1 month and 20 days
H2O_cluster_name:,H2O_from_python_gerd_trbony
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,2.867 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"accepting new members, healthy"


In [3]:
data = h2o.import_file('http://h2o-public-test-data.s3.amazonaws.com/smalldata/airlines/allyears2k_headers.zip')
train, valid, test = data.split_frame([0.8, 0.1], seed=69)

allColumns = data.columns
allColumns.remove("ArrDelay")
allColumns.remove("DepDelay")
allColumns.remove("TailNum")
allColumns.remove("CarrierDelay")
allColumns.remove("WeatherDelay")
allColumns.remove("NASDelay")
allColumns.remove("SecurityDelay")
allColumns.remove("LateAircraftDelay")
allColumns.remove("IsDepDelayed")
allColumns.remove("IsArrDelayed")
allColumns.remove("ActualElapsedTime")
allColumns.remove("ArrTime")

targetColumn = "IsArrDelayed"

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [4]:
allColumns

['Year',
 'Month',
 'DayofMonth',
 'DayOfWeek',
 'DepTime',
 'CRSDepTime',
 'CRSArrTime',
 'UniqueCarrier',
 'FlightNum',
 'CRSElapsedTime',
 'AirTime',
 'Origin',
 'Dest',
 'Distance',
 'TaxiIn',
 'TaxiOut',
 'Cancelled',
 'CancellationCode',
 'Diverted']

In [5]:
grid = h2o.grid.H2OGridSearch(H2ODeepLearningEstimator(epochs=40, hidden = [400, 400], activation = "RectifierWithDropout"),
                                search_criteria={"strategy":"RandomDiscrete", "max_models":12},
                                hyper_params={"seed":77, "l1":[0, 1e-6, 3e-6, 1e-5], "l2":[0, 1e-6, 3e-6, 1e-5],
                                             "input_dropout_ratio":[0, 0.1, 0.2, 0.3], #drops random n percent of input neurons
                                             "hidden_dropout_ratios":[[0,0], [0.2, 0.2], [0.4, 0.4], [0.6, 0.6]] #drops random n percent of hidden neurons
                                             },
                                grid_id="deeplearning-grid",
               )

grid.train(x = allColumns, y =targetColumn, training_frame = train, validation_frame = valid)


deeplearning Grid Build progress: |███████████████████████████████████████| 100%


In [32]:
grid.logloss()

{'deeplearning-grid_model_10': 0.27193525192836193,
 'deeplearning-grid_model_5': 0.31450255234047314,
 'deeplearning-grid_model_1': 0.4163576959016033,
 'deeplearning-grid_model_11': 0.4338300698471096,
 'deeplearning-grid_model_8': 0.463399256900399,
 'deeplearning-grid_model_3': 0.5114783362709405,
 'deeplearning-grid_model_9': 0.49603050130521037,
 'deeplearning-grid_model_6': 0.5388717188886684,
 'deeplearning-grid_model_2': 0.5360269731411931,
 'deeplearning-grid_model_7': 0.531030483727865,
 'deeplearning-grid_model_4': 0.5436650272373399,
 'deeplearning-grid_model_12': 0.5658208897902859}

In [35]:
grid[0].summary()


Status of Neuron Layers: predicting IsArrDelayed, 2-class classification, bernoulli distribution, CrossEntropy loss, 281,202 weights/biases, 3.2 MB, 1,423,403 training samples, mini-batch size 1


,,layer,units,type,dropout,l1,l2,mean_rate,rate_rms,momentum,mean_weight,weight_rms,mean_bias,bias_rms
0,,1,299,Input,0.0,,,,,,,,,
1,,2,400,RectifierDropout,20.0,0.0,0.00001,0.038699,0.153276,0.0,-0.004861,0.092402,-0.106496,0.172769
2,,3,400,RectifierDropout,20.0,0.0,0.00001,0.153406,0.205709,0.0,-0.04407,0.100449,0.249425,0.277373
3,,4,2,Softmax,,0.0,0.00001,0.013695,0.023256,0.0,-0.005293,0.248658,-0.021618,0.207327


In [39]:
#saving a model to local disk
best_model = h2o.get_model(grid.model_ids[0])
h2o.save_model(best_model, './')

'/home/gerd/Desktop/git/h2o-coursera/week4/deeplearning-grid_model_10'

In [41]:
h2o.cluster().shutdown()

H2O session _sid_bcef closed.
